In [2]:
ls ./../data/raw    

2022-10-13/  file_log_2022-10-13.pickle


In [3]:
import datetime
import hashlib
import logging
import pickle
import re
import unicodedata
from itertools import groupby
from pathlib import Path
from re import Pattern
from typing import Final, NamedTuple, Optional
from urllib.parse import urljoin

import click
import numpy as np
import pandas as pd
import requests
import structlog
from bs4 import BeautifulSoup
from pandas import Series
from pydantic import FileUrl, HttpUrl
from requests import RequestException, Response
from structlog.stdlib import BoundLogger 
from tqdm.auto import tqdm

from data_type import FileInfo, LocalFileInfo

/home/anosillus/.pyenv/versions/3.10.7/envs/fc-check-okinawa/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!ls ./../data/raw

2022-10-13  file_log_2022-10-13.pickle


In [5]:
with open('./../data/raw/file_log_2022-10-13.pickle', 'rb') as f:
    mynewlist = pickle.load(f)

In [6]:
mynewlist = sorted(mynewlist, key=lambda x: x.info.name)

In [15]:
#mynewlist

In [7]:
a = []
for i in mynewlist:
    b = Path('./../data/raw/' + str(i.download_date) + '/' + i.info.name)
    a.append(b)

In [8]:
len(pd.read_excel(a[0], sheet_name=None, index_col=0).keys()) == 1

True

In [9]:
a[-1]

PosixPath('../data/raw/2022-10-13/202209.xls')

In [10]:
pd.read_excel(a[-1], sheet_name=None, index_col=0).keys() 

dict_keys(['概要', '２頁 ', '総数及び世帯数', '地域別人口', '市町村別人口', '移動報告世帯数', '総数', '総数（男）', '総数（女）', '日本人総数', '日本人（男）', '日本人（女）', '外国人総数', '外国人（男）', '外国人（女）', 'R2国調確報(2020 Population Census)'])

In [11]:
file_name = a[0]
raw_df = pd.read_excel(file_name, header=None)

In [15]:
cols = raw_df.columns

In [22]:
for side_n, col in enumerate(cols):
    for vert_n, i in enumerate(raw_df[col]):
        if i == "市 町 村 名":
            print(side_n, vert_n)
            print(raw_df.iloc[vert_n,side_n])

0 4
市 町 村 名


In [ ]:
for side_n, col in enumerate(cols):
    for vert_n, i in enumerate(raw_df[col]):
        if i == "市 町 村 名":
            print(side_n, vert_n)
            print(raw_df.iloc[vert_n,side_n])

In [34]:
def cleaning_jp(fucking_jp:Series):
    return fucking_jp.fillna("").str.replace("\u3000", "").str.replace(" ", "").str.replace("現在", "当時").str.replace("県計", "")

In [35]:
def read_old_format(file_name):
    raw_df = pd.read_excel(file_name, header=None).replace('', np.nan).replace(' ', np.nan).dropna(axis=0, how='all')

    def clean_and_fill(col: int):
        return cleaning_jp(raw_df.iloc[col, :]).fillna(method="ffill")

    primary_columns = clean_and_fill(2)
    secondary_columns = clean_and_fill(3)
    third_columns = [
        a + b if b else a for a, b in zip(clean_and_fill(4), clean_and_fill(5))
    ]

    values = (
        raw_df.iloc[8:, 1:]
        .applymap(
            lambda x: unicodedata.decimal("NFD", cleaning_jp(x))
            if isinstance(x, str)
            else x
        )
        .astype("int")
    )
    df = values
    df.index = cleaning_jp(raw_df.iloc[8:, 0])
    df.index.name = "市町村名"
    df.columns = pd.MultiIndex.from_arrays(
        [primary_columns, secondary_columns, third_columns],
        names=("Primary", "Secondary", "Tertiary"),
    )[1:]

    return df

In [39]:
pd.set_option("display.width", 40)
RAW_DATA_PATH: Final[Path] = Path("./../data/raw/")

with open("./../data/raw/file_log_2022-10-13.pickle", "rb") as f:
    local_file_info = pickle.load(f)

file_names = [
    RAW_DATA_PATH / str(i.download_date) / i.info.name for i in local_file_info
]

In [40]:
rest = []

for file_name in file_names:
    print(file_name)
    # print(pd.read_excel(file_name, sheet_name=None, index_col=0).keys())

    if len(pd.read_excel(file_name, sheet_name=None, index_col=0).keys()) == 1:
        print(file_name)
        df = read_old_format(file_name)
        print(df.head(2))
    else:
        rest.append(file_name)



../data/raw/2022-10-13/199601.xls
../data/raw/2022-10-13/199601.xls
Primary      1月1日    当時人口          \
Secondary      当時                   
Tertiary  世帯数(世帯)      総数       男   
市町村名                                
那覇市        104794  298768  143714   
石川市          6779   21801   10909   

Primary               自然動態            \
Secondary                              
Tertiary        女  総数   出生   死亡 自然増加   
市町村名                                   
那覇市        155054 -61  311  101  210   
石川市         10892  -1   12   12    0   

Primary   社会動態                     \
Secondary   転入                 転出   
Tertiary    県外   県内 その他    計   県外   
市町村名                                
那覇市        370  562  17  949  355   
石川市         19   62   6   87   20   

Primary                        
Secondary                  社会  
Tertiary    県内 その他     計   増加  
市町村名                           
那覇市        815  50  1220 -271  
石川市         65   3    88   -1  
../data/raw/2022-10-13/199602.xls
../data/raw/2022-10

In [49]:
name = rest[0]

In [46]:
def call_sheet(name):
    return pd.read_excel(name, sheet_name=None, index_col=0).keys()

In [47]:
call_sheet(rest[0])

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'])

In [52]:
pd.read_excel(name, sheet_name='1').replace('', np.nan).replace(' ', np.nan).dropna(axis=0, how='all')

,Unnamed: 0,Unnamed: 1,市 町 村 別 人 口 総 数 及 び 世 帯 数 （平成 9年 5月 1日）,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,単位 : 人,Unnamed: 17
1,NaN,平成 9年,平 成 9 年 5月 1 日,NaN,NaN,NaN,NaN,NaN,平 成 9 年 4 月 中 の 人 口 増 加,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5月 1日,現 在 人 口,NaN,NaN,NaN,自 然 動 態,NaN,NaN,社 会 動 態,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,市 町 村 名,現 在,NaN,NaN,NaN,NaN,NaN,NaN,NaN,転 入,NaN,NaN,NaN,転 出,NaN,NaN,NaN,社会
4,NaN,世 帯 数,総 数,男,女,総 数,出 生,死亡,自然,県外,県内,その,計,県外,県内,その,計,増加
5,NaN,(世 帯),NaN,NaN,NaN,NaN,NaN,NaN,増加,NaN,NaN,他,NaN,NaN,NaN,他,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,伊 良 部 町,2242,6896,3393,3503,-14,2,5,-3,18,27,3,48,29,30,0,59,-11
64,多 良 間 村,506,1351,734,617,12,1,1,0,0,28,0,28,2,14,0,16,12
65,八重山郡,2260,5281,2706,2575,-2,5,1,4,40,137,0,177,30,153,0,183,-6
66,竹 富 町,1572,3510,1798,1712,22,3,1,2,34,110,0,144,22,102,0,124,20


In [58]:
read(pd.read_excel(name, sheet_name='3'))

Primary,５月１日,４月１日,増減数,増減率,５月１日,増減数,増減率
Secondary,,,,,,,
Tertiary,市部計那覇市,市部計那覇市,市部計那覇市,市部計那覇市,市部計那覇市,市部計那覇市,市部計那覇市
市町村名,,,,,,,
宜野湾市,83593,83270,323,0,82952,641,0
平良市,33323,33015,308,0,33099,224,0
石垣市,41949,41528,421,1,41745,204,0
浦添市,98155,97853,302,0,96950,1205,1
名護市,54387,53981,406,0,54106,281,0
糸満市,53931,53773,158,0,53643,288,0
沖縄市,117184,116941,243,0,116027,1157,1


In [70]:
df = pd.DataFrame({'species': ['bear', 'bear', 'marsupial'],
                  'population': [1864, 22000, 80000]},
                  index=['panda', 'polar', 'koala'])
df




for label, content in df.items():
    print(f'label: {label}')
    print(f'content: {content}', sep='\n')

label: species
content: panda         bear
polar         bear
koala    marsupial
Name: species, dtype: object
label: population
content: panda     1864
polar    22000
koala    80000
Name: population, dtype: int64


In [69]:
pd.read_excel(name, sheet_name='1')

,Unnamed: 0,Unnamed: 1,市 町 村 別 人 口 総 数 及 び 世 帯 数 （平成 9年 5月 1日）,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,単位 : 人,Unnamed: 17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,平成 9年,平 成 9 年 5月 1 日,NaN,NaN,,NaN,NaN,平 成 9 年 4 月 中 の 人 口 増 加,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5月 1日,現 在 人 口,NaN,NaN,,自 然 動 態,NaN,NaN,社 会 動 態,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,市 町 村 名,現 在,,NaN,NaN,,NaN,NaN,NaN,転 入,NaN,NaN,NaN,転 出,NaN,NaN,NaN,社会
4,NaN,世 帯 数,総 数,男,女,総 数,出 生,死亡,自然,県外,県内,その,計,県外,県内,その,計,増加
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
for i in pd.read_excel(name, sheet_name='1'):
    print(i[0])

U
U
 
U
U
U
U
U
U
U
U
U
U
U
U
U
単
U


In [62]:
read(pd.read_excel(name, sheet_name='1'),1).head(1)

Primary      5月1日   当時人口         \
Secondary      当時                 
Tertiary  世帯数(世帯)     総数      男   
市町村名                              
石川市          6830  21944  10971   

Primary               自然動態          \
Secondary                            
Tertiary       女   総数   出生 死亡 自然増加   
市町村名                                 
石川市        10973  106   26  9   17   

Primary   社会動態                        \
Secondary   転入                転出       
Tertiary    県外   県内 その他    計  県外  県内   
市町村名                                   
石川市         64  160   1  225  52  83   

Primary                 
Secondary           社会  
Tertiary  その他    計  増加  
市町村名                    
石川市         1  136  89

In [1]:
name 

NameError: name 'name' is not defined

In [63]:
pd.read_excel(name, sheet_name='1')

,Unnamed: 0,Unnamed: 1,市 町 村 別 人 口 総 数 及 び 世 帯 数 （平成 9年 5月 1日）,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,単位 : 人,Unnamed: 17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,平成 9年,平 成 9 年 5月 1 日,NaN,NaN,,NaN,NaN,平 成 9 年 4 月 中 の 人 口 増 加,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5月 1日,現 在 人 口,NaN,NaN,,自 然 動 態,NaN,NaN,社 会 動 態,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,市 町 村 名,現 在,,NaN,NaN,,NaN,NaN,NaN,転 入,NaN,NaN,NaN,転 出,NaN,NaN,NaN,社会
4,NaN,世 帯 数,総 数,男,女,総 数,出 生,死亡,自然,県外,県内,その,計,県外,県内,その,計,増加
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
def read(df,col_start=2):
    raw_df = df.replace('', np.nan).replace(' ', np.nan).dropna(axis=0, how='all')

    def clean_and_fill(col: int):
        return cleaning_jp(raw_df.iloc[col, :]).fillna(method="ffill")

    primary_columns = clean_and_fill(col_start)
    secondary_columns = clean_and_fill(col_start+1)
    third_columns = [
        a + b if b else a for a, b in zip(clean_and_fill(col_start+2), clean_and_fill(col_start+3))
    ]

    values = (
        raw_df.iloc[8:, 1:]
        .applymap(
            lambda x: unicodedata.decimal("NFD", cleaning_jp(x))
            if isinstance(x, str)
            else x
        )
        .astype("int")
    )
    df = values
    df.index = cleaning_jp(raw_df.iloc[8:, 0])
    df.index.name = "市町村名"
    df.columns = pd.MultiIndex.from_arrays(
        [primary_columns, secondary_columns, third_columns],
        names=("Primary", "Secondary", "Tertiary"),
    )[1:]

    return df